<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#INIT" data-toc-modified-id="INIT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>INIT</a></span></li><li><span><a href="#Compute-contact-distances" data-toc-modified-id="Compute-contact-distances-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Compute contact distances</a></span></li><li><span><a href="#Do-spectral-clustering" data-toc-modified-id="Do-spectral-clustering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Do spectral clustering</a></span></li><li><span><a href="#Extract-cluster" data-toc-modified-id="Extract-cluster-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extract cluster</a></span></li><li><span><a href="#Visualize-results" data-toc-modified-id="Visualize-results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Visualize results</a></span><ul class="toc-item"><li><span><a href="#With-VMD-script" data-toc-modified-id="With-VMD-script-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>With VMD script</a></span></li><li><span><a href="#Cluster-indices-per-frame" data-toc-modified-id="Cluster-indices-per-frame-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Cluster indices per frame</a></span></li></ul></li></ul></div>

# INIT

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging
import sys
logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
    format='%(asctime)s %(name)s-%(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')
import subprocess
import mdtraj as md
from argparse import ArgumentParser
import numpy as np
import matplotlib
# matplotlib.use('TkAgg')  #select different GUI backend for interactive plots
import matplotlib.pyplot as plt
plt.style.use("seaborn-colorblind")
import os
import math
import itertools
from shutil import copyfile
from helpfunc import *
from colvars import *
sys.path.append('MD_common/')
import MD_fun
from MD_spectral_clustering import SpectralClustering
from MD_extract_clusters import ExtractClusters
import nbimporter
import AnalyzeClusteredFrames as ancf
os.chdir(get_project_path())

logger = logging.getLogger("clustertrajs")


def load_traj(inputdir, traj_names, top_names, strides=None):
    traj = None
    for i, name in enumerate(traj_names):
        trajpath = inputdir + name
        stride=1 if strides is None else strides[i]
        if len(top_names) == 0 or top_names[i] is None:
            t = md.load(
                trajpath,
                stride=stride)
        else:
            t = md.load(
                trajpath,
                top=inputdir + top_names[i],
                stride=stride)
        traj = t if traj is None else traj + t
    return traj


outdir = "Result_Data/beta2-dror/clustering/"
trajtype = "DROR-A" 
dt = 5
traj_names, top_names = [], []
strides = None
inputdir = "DESRES-Trajectory_pnas2011b-A-00-no-water-no-lipid/pnas2011b-A-00-no-water-no-lipid/"
outdir = inputdir + "clustering/"
strides = [1]
traj_names, top_names = ["all.dcd"], ["all.pdb"]
dt = 10
strides = [1]
fulltrajname="SHOULDNOTSAVE"
cluster_outdir = outdir + "clustering/"
traj = load_traj(inputdir, traj_names, top_names, strides)
    
logger.debug(
    "Done. inputdir=%s, Loaded trajectory %s from %s trajectories. fulltrajname=%s, strides=%s, dt=%s ",
    inputdir, traj, len(traj_names), fulltrajname, strides, dt)

# Compute contact distances

In [ ]:
def compute_CAs(traj, input_args, dt=1):
    fun = MD_fun.MD_functions()
    fun.initialize_trajectory(ArgumentParser(), input_args=input_args)
    
    #distance functions
    spectral_norm = lambda x, y, i, j: np.linalg.norm(x - y, ord=2)
    frobenius_norm = lambda x, y, i, j: np.linalg.norm(x - y, ord='fro')

    process_count = 7
    #mkl.set_num_threads(process_count)
    frame_to_frame_dists, all_frame_contact_matrices = fun.computeFrameToFrameCalphaContacts(
        traj[::dt], async=False, computeContactNorm=spectral_norm, process_count=process_count)
    frame_norms = [np.linalg.norm(m) for m in all_frame_contact_matrices]
    plt.hist(frame_norms)
    plt.show()
    return frame_to_frame_dists, all_frame_contact_matrices


logger.debug("Started computing distances for a traj with %s frames and dt=%s",
             len(traj), dt)

compute_CAs(traj, ["-od", outdir], dt=dt)

logger.debug("Done")

# Do spectral clustering

In [ ]:
frame_to_frame_filename = "frame_to_frame_CA_contacts_.txt"

logger.info("Loading cluster file from %s/%s", outdir, frame_to_frame_filename)
clustering = SpectralClustering()
clustering.initialization(
    ArgumentParser(),
    input_args=[
        "-id", outdir, "-d", frame_to_frame_filename, "-od",
        cluster_outdir, "-cdist"
    ])
clustering.cluster()
logger.debug("Done")

# Extract cluster

In [ ]:
# cluster_outdir = outdir + "clustering_7_clusters/"

clusterer = ExtractClusters()
clusterer.main(
    ArgumentParser(),
    ["-ind", cluster_outdir + "cluster_indices_.txt", "-od", cluster_outdir],
    input_traj=traj[::dt])
cluster_frames_path = cluster_outdir + "clustered_frames/"
#copy toplogy file for convencience when looking at the frames
# copyfile(inputdir + top_names[0],cluster_frames_path + top_names[0])
logger.debug("Done")

# Visualize results

## With VMD script

In [ ]:
def run_bashscript(scriptpath):
    subprocess.call(scriptpath)


def run_bash(command):
    logger.info("Running bash command:\n%s", command)
    return subprocess.call(command.split())
vmd_bash = ("topology=%s\n" \
    +"cmd=\"vmd\"\n"\
    +"for f in reps_cluster_*\n"\
    +"do\n"\
    +"    cmd+=\" -f $topology $f\"\n"\
    +"done\n"\
    +"echo $cmd\n"\
    +"$cmd\n")%(inputdir + top_names[0])
# run_bash("sleep 2")
vmdscript_path = cluster_frames_path + "openvmd.sh"
with open(vmdscript_path, "wr") as bash_script:
    bash_script.write(vmd_bash)
# run_bash(vmd_bash)
logger.debug("Done")
# subprocess.call(vmdscript_path)

## Cluster indices per frame

In [ ]:
cluster_simu = Simulation({
    "condition": "NA",
    "number": "NA",
    "name": "all",
    "stride": dt
})

cluster_simu.clusterpath = cluster_outdir
cluster_simu.traj = traj[::dt]
# cluster_simu.timestep = 0 #NA
cluster_simu = ancf.load_cluster_indices(cluster_simu)

plt.figure(figsize=(16,4))
plt.plot(cluster_simu.cluster_indices)
plt.grid()
plt.xlabel("Frame #")
plt.ylabel("Cluster index")
plt.show()